##### Import Package

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from konlpy.tag import Twitter

import pandas as pd
import pickle

##### Get Article

In [2]:
df_list = []

article_df = ""

with open("./data/article_2016-06-01.plk", 'rb') as file:
    article_df = pickle.load(file)

len(article_df), article_df.columns

(2327,
 Index(['newsid', 'oid', 'newspaper', 'title', 'link', 'comment', 'likeit',
        'content', 'date', 'category'],
       dtype='object'))

##### Split Tratin, Test

In [3]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(2094, 233, 2094, 233)

##### Set Morpheme

In [4]:
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

##### Set Model

In [5]:
clf = Pipeline([
#             ('vect', TfidfVectorizer(tokenizer=tokenize_pos, stop_words=stop_words, ngram_range=(1,2))),
            ('vect', TfidfVectorizer()),
            ('clf', MultinomialNB(alpha=0.01)),
        ])

In [6]:
%time model = clf.fit(X_train,y_train) 

CPU times: user 1.43 s, sys: 65.5 ms, total: 1.49 s
Wall time: 1.5 s


##### Confusion Matrix, Classfication Report
- category - 0:정치, 1:경제, 2:사회, 3.생활/문화, 4.세계, 5.IT/과학

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%time y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

CPU times: user 110 ms, sys: 7.3 ms, total: 118 ms
Wall time: 118 ms
[[41  0  3  1  1  0]
 [ 1 33  0  5  1  1]
 [ 1  2 77  2  1  0]
 [ 0  0  2 18  0  1]
 [ 1  1  2  0 25  0]
 [ 0  1  1  0  0 11]]
             precision    recall  f1-score   support

          0       0.93      0.89      0.91        46
          1       0.89      0.80      0.85        41
          2       0.91      0.93      0.92        83
          3       0.69      0.86      0.77        21
          4       0.89      0.86      0.88        29
          5       0.85      0.85      0.85        13

avg / total       0.88      0.88      0.88       233



In [8]:
X_test = X_test.reset_index(drop=True)

In [9]:
# 0:정치, 1:경제, 2:사회, 3.생활/문화, 4.세계, 5.IT/과학
classification_dict = {
    0:"정치",
    1:"경제",
    2:"사회",
    3:"생활/문화",
    4:"세계",
    5:"IT/과학",
}
result = model.predict([ X_test[0], X_test[1], X_test[2] ])
for idx, category in enumerate(result):
    print(classification_dict[category], X_test[idx][:80])

경제 인천 송도 삼성바이오로직스 현장을 가다“삼성이 처음 핸드폰 사업을 시작할 때 망치로 핸드폰을 깨부수는 등 온갖 실험을 했던 동영상도 보여주며 ‘
사회 [구의역 사고를 보며] 청년을 위한 나라는 없다 [ 주동식 지역평등시민연대 대표] 요즘 사람의 실제 나이는 전통적인 개념의 숫자에 0.8을 곱해
정치  [한겨레] 구의역·남양주 사고에 입법대책 분주더민주, 직접 고용법 제정안 발의국민의당, 청년고용할당제 확대정의당도 ‘기업살인법’ 발의키로정부·


##### save model

In [10]:
import pickle

pickle.dump(model, open("./models/classification_model.plk", "wb"))

In [11]:
load_model = pickle.load(open("./models/classification_model.plk", "rb"))

In [12]:
test_str1 = "네이버와 카카오 드론 기술 발전에 주력"
test_str2 = "요즘에 환율과 주가가 예측 불허"
print( classification_dict[ load_model.predict([test_str1])[0] ] )
print( classification_dict[ load_model.predict([test_str2])[0] ] )

IT/과학
경제
